### Подготовка датасета по показателю объем убоя верблюда

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from pmdarima import auto_arima
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

from pylab import rcParams
from IPython.display import display
import math
from prophet import Prophet
pd.set_option('display.max_columns', 130)


import warnings
from statsmodels.tools.sm_exceptions import InterpolationWarning
warnings.simplefilter("ignore", category=InterpolationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)



In [2]:
df = pd.read_excel("../../Data cleansing/output data/Просуммированные по категориям с доп регрессорами.xlsx")
df.head(5)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Верблюды,АКМОЛИНСКАЯ ОБЛАСТЬ,0.00,0.00,0.40,0.00,0.00,0.00,0.09,1.00,0.00,0.00,0.20,0.00,0.00,0.18,0.28,0.00,0.00,0.40,0.00,0.00,0.65,0.00,0.31,1.00,0.00,0.00,0.00,0.00,0.00,2.01,0.00,1.20,0.00,0.00,2.18,0.39,0.00,0.00,1.04,0.00,0.14,2.08,0.00,0.00,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.66,0.00,0.33,0.00,0.9,0.00,0.00,0.00,10.08,0.00,0.0,0.00,0.0,0.00,0.54,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.36,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.00,0.40,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00
1,Верблюды,АКТЮБИНСКАЯ ОБЛАСТЬ,101.98,67.47,374.84,115.59,218.72,14.15,19.77,3.00,39.16,46.16,238.56,463.35,109.04,72.94,384.96,114.35,221.89,10.93,18.79,3.50,38.53,46.64,216.08,472.07,110.32,68.17,380.15,127.12,217.61,14.98,21.26,7.14,44.63,51.78,239.43,513.33,115.70,69.40,371.53,130.17,218.41,14.59,21.76,7.19,45.97,55.48,252.68,527.66,117.94,70.37,385.75,117.80,218.45,15.54,21.78,7.2,47.06,58.02,257.54,543.35,119.80,71.3,396.90,121.8,228.10,15.90,21.9,7.2,63.00,59.90,260.20,552.10,121.60,71.8,398.40,128.50,228.80,13.10,22.50,7.20,63.90,61.90,150.70,554.90,121.30,73.3,404.20,128.70,234.90,13.60,23.60,6.3,49.80,62.4,153.70,552.8,118.07,69.17,384.21,121.01,223.46,11.76,21.39,5.52,48.18,61.39,150.87,531.79,119.50,72.60,391.0,126.1,229.10,12.2,24.10,5.7,49.87,63.00,154.3,538.40
2,Верблюды,АЛМАТИНСКАЯ ОБЛАСТЬ,1.00,0.20,51.60,25.40,0.00,61.90,76.87,95.57,16.09,0.10,10.80,46.78,15.06,13.70,131.80,9.20,17.89,130.12,2.00,0.00,4.99,2.06,4.90,44.48,15.06,23.94,41.60,19.54,3.00,15.14,18.50,14.82,14.52,12.60,9.92,43.11,20.72,2.96,15.24,0.00,1.00,20.03,3.44,7.77,117.25,1.00,0.00,20.34,21.95,3.04,17.91,26.81,9.16,18.80,13.37,0.0,17.29,15.90,4.69,23.10,10.85,9.7,42.75,0.0,6.21,21.10,4.0,2.6,28.10,4.53,79.50,93.18,23.70,11.2,16.00,2.00,3.10,19.62,2.30,4.85,11.18,2.80,9.65,36.44,10.60,11.5,26.10,7.20,12.75,11.20,11.20,27.0,25.26,17.9,21.90,12.5,16.55,16.81,22.78,12.10,12.44,6.85,39.65,17.22,8.85,27.48,0.48,29.72,18.90,17.40,12.5,16.4,11.60,16.7,15.70,24.0,7.70,6.90,5.4,12.60
3,Верблюды,АТЫРАУСКАЯ ОБЛАСТЬ,213.89,167.70,306.60,164.97,342.57,192.00,43.60,113.03,262.37,193.97,308.17,1087.33,325.10,190.60,301.10,154.84,328.50,220.30,66.10,118.20,234.90,196.51,270.92,974.18,303.30,167.41,323.54,182.34,349.10,249.40,57.57,88.40,221.21,199.63,278.80,964.02,305.22,159.62,326.87,159.50,367.20,257.50,60.01,78.10,253.27,332.00,355.05,989.86,280.08,165.05,334.51,134.85,367.97,321.22,126.22,93.7,263.10,366.66,348.54,1041.38,293.40,154.8,339.44,143.6,405.80,308.35,88.5,130.5,575.30,458.70,357.62,998.10,292.76,188.3,487.02,129.28,421.44,351.20,95.77,97.54,624.80,631.65,499.30,1179.66,286.67,189.8,513.16,171.46,402.10,432.22,121.53,126.2,637.50,651.7,482.94,1412.2,307.85,200.59,466.60,182.63,414.56,405.19,184.80,154.96,717.92,719.01,522.83,1164.77,323.81,423.76,441.9,219.1,487.69,446.9,191.43,133.5,730.34,709.68,537.6,930.28
4,Верблюды,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,0.30,0.00,0.00,1.14,6.12,4.70

In [3]:
df['Показатель'].unique()

array(['Верблюды', 'КРС', 'Лошади', 'Молоко', 'Овцы и козы', 'Птица',
       'Свиньи', 'Яйца', 'Температура', 'Поголовье: КРС',
       'Поголовье: лошади', 'Поголовье: овцы и козы', 'Поголовье: свиньи',
       'Поголовье: птица домашняя', 'Поголовье: верблюды', 'Осадки',
       'Цена: Говядина', 'Цена: Баранина', 'Цена: Молоко', 'Цена: Яйца'],
      dtype=object)

In [4]:
df_camel = df[df['Показатель'].isin(['Верблюды', 'Температура', 'Осадки', 'Поголовье: верблюды'])]
df_camel.sample(10)

,Показатель,Регион,2015-01,2015-02,2015-03,2015-04,2015-05,2015-06,2015-07,2015-08,2015-09,2015-10,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08,2016-09,2016-10,2016-11,2016-12,2017-01,2017-02,2017-03,2017-04,2017-05,2017-06,2017-07,2017-08,2017-09,2017-10,2017-11,2017-12,2018-01,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01,2020-02,2020-03,2020-04,2020-05,2020-06,2020-07,2020-08,2020-09,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03,2021-04,2021-05,2021-06,2021-07,2021-08,2021-09,2021-10,2021-11,2021-12,2022-01,2022-02,2022-03,2022-04,2022-05,2022-06,2022-07,2022-08,2022-09,2022-10,2022-11,2022-12,2023-01,2023-02,2023-03,2023-04,2023-05,2023-06,2023-07,2023-08,2023-09,2023-10,2023-11,2023-12,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
326,Осадки,ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,29.40,33.40,41.20,50.60,63.80,13.70,35.60,56.80,62.60,46.80,61.2,32.10,20.40,9.50,24.20,48.60,34.50,111.10,185.60,21.50,27.00,61.50,90.70,49.80,4.50,15.70,11.60,55.70,18.00,37.9,94.80,20.40,58.50,34.30,16.00,16.90,4.10,28.60,103.10,41.50,62.10,12.40,46.40,84.00,43.90,56.70,60.30,14.30,36.00,16.10,5.90,53.00,67.30,119.30,41.80,29.60,19.80,87.00,65.70,38.80,26.20,60.80,13.90,19.20,6.20,32.90,61.8,45.1,56.30,66.70,20.90,32.50,19.70,39.0,64.1,3.70,10.30,45.80,52.70,47.30,13.50,57.20,51.40,30.30,32.30,12.10,57.80,16.90,14.70,57.20,30.1,36.20,5.90,31.10,73.50,44.30,22.10,39.70,31.70,27.20,37.60,6.10,35.90,76.90,75.60,107.60,63.90,46.50,34.80,53.50,16.70,36.50,64.40,55.10,120.60,50.60,28.60,24.10,50.70,48.30
14,Верблюды,ОБЛАСТЬ АБАЙ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.30,0.0,0.00,0.00,0.00,5.70,24.40,0.00,0.00,0.00,0.00,0.00,2.10,0.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00,0.00,0.00,0.00,1.68,0.00,0.00,0.00,0.70,1.40,17.10
17,Верблюды,ПАВЛОДАРСКАЯ ОБЛАСТЬ,0.00,0.00,0.00,0.96,0.00,0.00,0.00,0.00,1.60,0.00,0.0,0.00,0.00,0.00,0.00,1.86,0.00,1.20,0.00,0.37,0.12,0.00,0.00,0.00,0.00,0.75,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,1.01,0.80,0.00,0.00,0.74,0.00,0.00,0.00,0.30,0.00,0.00,0.00,0.38,0.56,0.00,1.00,0.00,0.94,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.43,0.00,0.00,1.00,0.00,0.00,0.0,0.0,0.00,0.00,1.53,5.70,0.00,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.42,0.00,0.00,0.42,0.00,0.00,0.00,0.00,0.00,0.40,0.00,0.00,0.00,0.00
327,Осадки,ГАСТАНА,21.40,9.30,27.60,25.40,113.20,53.30,24.00,13.10,11.00,18.10,38.8,40.60,21.00,7.80,22.30,33.40,11.80,73.60,105.10,3.90,26.90,30.30,32.50,48.80,18.60,18.70,14.90,29.90,20.00,15.9,26.30,5.90,16.20,34.60,21.50,32.70,15.10,7.80,35.00,49.30,36.80,47.00,42.40,73.60,11.60,55.80,42.40,12.50,10.40,25.50,23.20,21.10,11.40,63.70,13.30,27.40,35.90,12.70,47.70,39.60,52.00,45.10,7.30,44.70,13.50,96.30,52.8,38.8,27.30,21.30,49.30,12.20,10.80,31.3,52.3,4.30,10.80,21.60,44.00,7.60,52.60,22.80,51.30,22.90,25.00,19.10,31.30,7.10,9.90,18.40,33.0,26.10,15.20,17.10,47.00,19.30,27.30,19.60,20.20,48.90,0.60,24.00,18.30,27.90,63.80,23.00,35.40,61.80,43.90,30.20,21.00,21.60,77.70,46.70,28.90,114.30,35.60,40.50,38.00,30.40
313,Поголовье: верблюды,РЕСПУБЛИКА КАЗАХСТАН,163951.00,164483.00,181835.00,188322.00,191195.00,190604.00,179709.00,178305.00,177020.00,175991.00,173364.0,169458.00,168071.00,167948.00,185523.00,192835.00,196472.00,195485.00,184489.00,183330.00,181165.00,179585.00,176753.00,172469.00,176906.00,177563.00,195011.00,201962.00,205787.00,205375.0,198626.00,197512.00,195287.00,193623.0

In [6]:
# Step 1: Pivot to wide format (each indicator becomes columns of periods)
df_wide = df_camel.pivot(index="Регион", columns="Показатель")

# Step 2: Flatten multi-level columns: ('2015-01', 'КРС') → 'КРС_2015-01'
df_wide.columns = [f"{col[1]}_{col[0]}" for col in df_wide.columns]
df_wide = df_wide.reset_index()

# Step 3: Melt: one row per region-period-indicator
df_melted = df_wide.melt(id_vars="Регион", var_name="indicator_period", value_name="value")

# Step 4: Extract 'Период' and 'Показатель' from the combined column
df_melted["Период"] = df_melted["indicator_period"].str.extract(r"_(\d{4}-\d{2})$")
df_melted["Показатель"] = df_melted["indicator_period"].str.extract(r"^(.+)_\d{4}-\d{2}")

# Step 5: Pivot again to get final modeling format: one row per region+period, one column per indicator
df_camel = df_melted.pivot_table(index=["Регион", "Период"], columns="Показатель", values="value").reset_index()
print(df_camel.groupby("Регион").size().reset_index(name="Количество строк"))
df_camel

                            Регион  Количество строк
0              АКМОЛИНСКАЯ ОБЛАСТЬ               120
1              АКТЮБИНСКАЯ ОБЛАСТЬ               120
2              АЛМАТИНСКАЯ ОБЛАСТЬ               120
3               АТЫРАУСКАЯ ОБЛАСТЬ               120
4   ВОСТОЧНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
5                          ГАЛМАТЫ               120
6                          ГАСТАНА               120
7                         ГШЫМКЕНТ                79
8               ЖАМБЫЛСКАЯ ОБЛАСТЬ               120
9    ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ               120
10          КАРАГАНДИНСКАЯ ОБЛАСТЬ               120
11            КОСТАНАЙСКАЯ ОБЛАСТЬ               120
12          КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ               120
13           МАНГИСТАУСКАЯ ОБЛАСТЬ               120
14                    ОБЛАСТЬ АБАЙ                31
15                  ОБЛАСТЬ ЖЕТІСУ                31
16                  ОБЛАСТЬ ҰЛЫТАУ                31
17            ПАВЛОДАРСКАЯ ОБЛАСТЬ            

Показатель,Регион,Период,Верблюды,Осадки,Поголовье: верблюды,Температура
0,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-01,0.00,9.8,140.0,-12.490323
1,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-02,0.00,9.8,111.0,-10.192857
2,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-03,0.40,8.3,116.0,-5.870968
3,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-04,0.00,8.8,120.0,4.490000
4,АКМОЛИНСКАЯ ОБЛАСТЬ,2015-05,0.00,42.8,123.0,14.574194
...,...,...,...,...,...,...
2166,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-08,142.22,0.0,2309.0,27.874194
2167,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-09,138.52,0.5,2298.0,20.766667
2168,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-10,135.10,13.6,2293.0,13.200000
2169,ТУРКЕСТАНСКАЯ ОБЛАСТЬ,2024-11,262.18,12.1,2293.0,6.500000


In [7]:
df_camel = df_camel[df_camel["Регион"] != 'РЕСПУБЛИКА КАЗАХСТАН']
df_camel = df_camel.drop(columns=['Осадки', 'Поголовье: верблюды', 'Температура'])
df_camel.sample(10)

Показатель,Регион,Период,Верблюды
1477,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2021-07,0.80
218,АКТЮБИНСКАЯ ОБЛАСТЬ,2023-03,384.21
2000,СЕВЕРО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2017-05,0.00
371,АТЫРАУСКАЯ ОБЛАСТЬ,2015-12,1087.33
943,ЖАМБЫЛСКАЯ ОБЛАСТЬ,2017-01,55.49
1322,КОСТАНАЙСКАЯ ОБЛАСТЬ,2018-08,0.00
1286,КОСТАНАЙСКАЯ ОБЛАСТЬ,2015-08,0.00
174,АКТЮБИНСКАЯ ОБЛАСТЬ,2019-07,21.78
1109,ЗАПАДНО-КАЗАХСТАНСКАЯ ОБЛАСТЬ,2020-11,1.40
1429,КЫЗЫЛОРДИНСКАЯ ОБЛАСТЬ,2017-07,0.00


In [8]:
df_camel.to_excel("Датасет по верблюдам.xlsx", index=False)